Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [9]:
batch_size = 128
beta = .001

graph = tf.Graph()

with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
                                                                                beta * tf.nn.l2_loss(weights)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [10]:

num_steps = 3001
def run_training(graph, num_steps,train_dataset, train_labels):
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

In [11]:
run_training(graph,num_steps,train_dataset, train_labels)

Initialized
Minibatch loss at step 0: 19.560663
Minibatch accuracy: 16.4%
Validation accuracy: 13.3%
Minibatch loss at step 500: 2.806932
Minibatch accuracy: 77.3%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 1.788657
Minibatch accuracy: 78.9%
Validation accuracy: 78.0%
Minibatch loss at step 1500: 1.182526
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 0.800223
Minibatch accuracy: 84.4%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 0.739643
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.699217
Minibatch accuracy: 83.6%
Validation accuracy: 81.3%
Test accuracy: 88.4%


## Neural Network :

In [47]:
batch_size = 128
hidden_layer = 1024
beta = .001

g_NN = tf.Graph()
with g_NN.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer]))

    biases = tf.Variable(tf.zeros([hidden_layer]))

    weights1 = tf.Variable(tf.truncated_normal([hidden_layer, num_labels]))

    biases1 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    layer1_out = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
    logits = tf.matmul(layer1_out, weights1) + biases1

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) \
                                              + beta * tf.nn.l2_loss(weights) + beta * tf.nn.l2_loss(weights1)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    layer1_out_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
    logits_valid = tf.matmul(layer1_out_valid, weights1) + biases1
    valid_prediction = tf.nn.softmax(logits_valid)

    layer1_out_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
    logits_test = tf.matmul(layer1_out_test, weights1) + biases1
    test_prediction = tf.nn.softmax(logits_test)


run_training(g_NN,10001,train_dataset, train_labels)

Initialized
Minibatch loss at step 0: 665.415527
Minibatch accuracy: 6.2%
Validation accuracy: 29.2%
Minibatch loss at step 500: 197.188202
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 114.076729
Minibatch accuracy: 85.2%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 69.119957
Minibatch accuracy: 82.8%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 41.452358
Minibatch accuracy: 86.7%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 25.191143
Minibatch accuracy: 89.1%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 15.451426
Minibatch accuracy: 89.1%
Validation accuracy: 86.9%
Minibatch loss at step 3500: 9.647138
Minibatch accuracy: 85.2%
Validation accuracy: 87.5%
Minibatch loss at step 4000: 6.025967
Minibatch accuracy: 88.3%
Validation accuracy: 87.8%
Minibatch loss at step 4500: 3.910480
Minibatch accuracy: 84.4%
Validation accuracy: 88.2%
Minibatch loss at step 5000: 2.481124
Minibatch accuracy: 89.8%
Validatio

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [89]:
batch_size = 128
hidden_layer = 1024
beta = 0

g_NN = tf.Graph()
with g_NN.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer]))

    biases = tf.Variable(tf.zeros([hidden_layer]))

    weights1 = tf.Variable(tf.truncated_normal([hidden_layer, num_labels]))

    biases1 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    layer1_out = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
    logits = tf.matmul(layer1_out, weights1) + biases1

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) \
                                                                              + beta * tf.nn.l2_loss(weights)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    layer1_out_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
    logits_valid = tf.matmul(layer1_out_valid, weights1) + biases1
    valid_prediction = tf.nn.softmax(logits_valid)

    layer1_out_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
    logits_test = tf.matmul(layer1_out_test, weights1) + biases1
    test_prediction = tf.nn.softmax(logits_test)

run_training(g_NN,num_steps,train_dataset[:128*5,:], train_labels[:128*5,:])

Initialized
Minibatch loss at step 0: 337.734375
Minibatch accuracy: 14.8%
Validation accuracy: 29.4%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Test accuracy: 82.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [98]:
batch_size = 128
hidden_layer = 1024

g_NN = tf.Graph()
with g_NN.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer]))

    biases = tf.Variable(tf.zeros([hidden_layer]))

    weights1 = tf.Variable(tf.truncated_normal([hidden_layer, num_labels]))

    biases1 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    layer1_out = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases), .5)
    logits = tf.matmul(layer1_out, weights1) + biases1

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    layer1_out_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
    logits_valid = tf.matmul(layer1_out_valid, weights1) + biases1
    valid_prediction = tf.nn.softmax(logits_valid)

    layer1_out_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
    logits_test = tf.matmul(layer1_out_test, weights1) + biases1
    test_prediction = tf.nn.softmax(logits_test)
    
run_training(g_NN,10001,train_dataset, train_labels)

Initialized
Minibatch loss at step 0: 461.654663
Minibatch accuracy: 3.9%
Validation accuracy: 28.1%
Minibatch loss at step 500: 36.661930
Minibatch accuracy: 79.7%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 13.594918
Minibatch accuracy: 75.8%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 14.878325
Minibatch accuracy: 69.5%
Validation accuracy: 78.9%
Minibatch loss at step 2000: 12.121876
Minibatch accuracy: 76.6%
Validation accuracy: 79.0%
Minibatch loss at step 2500: 13.671824
Minibatch accuracy: 71.1%
Validation accuracy: 77.3%
Minibatch loss at step 3000: 12.944775
Minibatch accuracy: 74.2%
Validation accuracy: 79.4%
Minibatch loss at step 3500: 9.058495
Minibatch accuracy: 76.6%
Validation accuracy: 78.4%
Minibatch loss at step 4000: 4.096803
Minibatch accuracy: 72.7%
Validation accuracy: 79.2%
Minibatch loss at step 4500: 5.948316
Minibatch accuracy: 78.9%
Validation accuracy: 80.3%
Minibatch loss at step 5000: 2.086392
Minibatch accuracy: 78.1%
Validation 

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [57]:
batch_size = 128
layer1_size = 1024
layer2_size = 128
layer1_keep = .5
layer2_keep = .5
beta = 0.005
learning_rate_init = .1
g_NN = tf.Graph()
with g_NN.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, layer1_size]))

    biases = tf.Variable(tf.zeros([layer1_size]))

    weights1 = tf.Variable(tf.truncated_normal([layer1_size, layer2_size],stddev=.1))

    biases1 = tf.Variable(tf.zeros([layer2_size]))
    
    weights2 = tf.Variable(tf.truncated_normal([layer2_size, num_labels],stddev=.1))

    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    layer1_out = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases), layer1_keep)
    layer2_out = tf.nn.dropout(tf.nn.relu(tf.matmul(layer1_out, weights1) + biases1), layer2_keep)
    logits = tf.matmul(layer2_out, weights2) + biases2

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) \
                    + beta * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) )

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learning_rate_init, global_step, 10000, 0.90)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate_init).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    layer1_out_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
    layer2_out_valid = tf.nn.relu(tf.matmul(layer1_out_valid, weights1) + biases1)
    logits_valid = tf.matmul(layer2_out_valid, weights2) + biases2
    valid_prediction = tf.nn.softmax(logits_valid)

    layer1_out_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
    layer2_out_test = tf.nn.relu(tf.matmul(layer1_out_test, weights1) + biases1)    
    logits_test = tf.matmul(layer2_out_test, weights2) + biases2
    test_prediction = tf.nn.softmax(logits_test)
    
run_training(g_NN,15001,train_dataset, train_labels)

Initialized
Minibatch loss at step 0: 1602.499146
Minibatch accuracy: 13.3%
Validation accuracy: 10.5%
Minibatch loss at step 500: 948.673889
Minibatch accuracy: 36.7%
Validation accuracy: 51.2%
Minibatch loss at step 1000: 577.542114
Minibatch accuracy: 63.3%
Validation accuracy: 73.2%
Minibatch loss at step 1500: 353.000214
Minibatch accuracy: 60.9%
Validation accuracy: 76.4%
Minibatch loss at step 2000: 216.212006
Minibatch accuracy: 68.0%
Validation accuracy: 80.2%
Minibatch loss at step 2500: 133.040009
Minibatch accuracy: 68.8%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 82.058319
Minibatch accuracy: 75.0%
Validation accuracy: 82.4%
Minibatch loss at step 3500: 50.998260
Minibatch accuracy: 75.0%
Validation accuracy: 83.1%
Minibatch loss at step 4000: 31.851360
Minibatch accuracy: 75.0%
Validation accuracy: 83.8%
Minibatch loss at step 4500: 20.088167
Minibatch accuracy: 75.8%
Validation accuracy: 84.0%
Minibatch loss at step 5000: 12.758931
Minibatch accuracy: 78.1%
